In [3]:
from scamp import test_run, Session, wait
import random
import time
from pythonosc import udp_client
from src.config import ARTIFACTS_RELPATH
from src.scamp_utils import set_default_scamp_settings

set_default_scamp_settings()

Testing for working audio driver...
jackdmp 1.9.22
Copyright 2001-2005 Paul Davis and others.
Copyright 2004-2016 Grame.
Copyright 2016-2023 Filipe Coelho.
jackdmp comes with ABSOLUTELY NO WARRANTY
This is free software, and you are welcome to redistribute it
under certain conditions; see the file COPYING for details


ALSA lib conf.c:4005:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/opt/conda/envs/default/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default
fluidsynth: error: Failed to open the "default" audio device
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'coreaudio'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseaudio, sdl2
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'dsound'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseau

## Using Scamp's osc methods

In [53]:
def start_session():
    touch_session = Session()
    touch_osc_part = touch_session.new_osc_part(
        name="touch_osc",
        ip_address="touch_osc",
        port=9000,
    )
    return touch_session, touch_osc_part


def test_run(
    touch_osc_session, touch_osc_part, lowest, highest, step, intervals, repeat_count = 1
):
    touch_osc_session.tempo = 40
    touch_osc_session.start_transcribing()
    sequence = [
        *range(lowest, highest, step),
        *range(highest, lowest, -1 * step),
    ]
    print("Running sequence: ", sequence)
    for repeat in range(repeat_count):
        print("Repeat: ", repeat)
        for pitch in sequence:
            length = 0.1
            velocity = 110
            for interval in intervals:
                touch_osc_part.play_note(
                    pitch + interval, velocity, length, blocking=False
                )
            wait(length)

    tr = touch_osc_session.stop_transcribing()
    tr.save_to_json(f"{ARTIFACTS_RELPATH}/touch_osc.json")

In [54]:
touch_osc_session, touch_osc_part = start_session()

test_run(touch_osc_session, touch_osc_part, 50, 70, 3, [0, 4], 5)

Running sequence:  [50, 53, 56, 59, 62, 65, 68, 70, 67, 64, 61, 58, 55, 52]
Repeat:  0
Repeat:  1
Repeat:  2
Repeat:  3
Repeat:  4


## Using python osc

Scamps's osc feature seems to have been developed with only ScampUtils of supercollider in mind and does not communicate directly with touchosc's natural way of doing things. 

Until a proper script in lua (or scamp) is developed, python-osc helps with experimentation.


In [10]:
client = udp_client.SimpleUDPClient("touch_osc", 9000)

In [11]:
lowest = 50
highest = 60
sequence = [*range(lowest, highest, 2), *reversed(range(lowest, highest, 2))]
print(sequence)
for _ in range(1):
    for x in sequence:
        address = "/touch_osc"
        # pitch = x * 2 + 50
        pitches = [
            # x * 2 + 50 + 2,
            x
            # x * 2 + 40,
        ]
        # velocity = x * 10
        for pitch in pitches:
            client.send_message(f"{address}/start_note", [pitch, 127])
        time.sleep(0.3)
        for pitch in pitches:
            client.send_message(f"{address}/end_note", [pitch, 127])
        time.sleep(0.5)

[50, 52, 54, 56, 58, 58, 56, 54, 52, 50]
